## Hyperparameters Tuning
1. Model hyperparameters
2. Algorithm hyperparameters

In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [2]:
!pip install -q -U keras-tuner
import kerastuner as kt

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [5]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))
    
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate = hp_learning_rate),
                 loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])
    return model

In [6]:
tuner = kt.Hyperband(model_builder,
                    objective='val_accuracy',
                    max_epochs=10,
                    factor=3,
                    directory='my_dir',
                    project_name='intro_to_kt')

In [7]:
# Hyperparameter Search 전에 training step 마지막에 training output 클리어
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)

In [9]:
tuner.search(img_train, label_train, epochs=10, validation_data = (img_test, label_test), callbacks=[ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Epoch 1/4
1875/1875 [==============================] - ETA: 0s - loss: 2.7621 - accuracy: 0.09 - ETA: 2s - loss: 1.3426 - accuracy: 0.53 - ETA: 2s - loss: 1.0650 - accuracy: 0.62 - ETA: 2s - loss: 0.9620 - accuracy: 0.65 - ETA: 2s - loss: 0.8989 - accuracy: 0.67 - ETA: 2s - loss: 0.8384 - accuracy: 0.69 - ETA: 2s - loss: 0.8062 - accuracy: 0.71 - ETA: 2s - loss: 0.7721 - accuracy: 0.72 - ETA: 2s - loss: 0.7384 - accuracy: 0.73 - ETA: 2s - loss: 0.7143 - accuracy: 0.74 - ETA: 2s - loss: 0.6954 - accuracy: 0.74 - ETA: 1s - loss: 0.6802 - accuracy: 0.75 - ETA: 1s - loss: 0.6662 - accuracy: 0.75 - ETA: 1s - loss: 0.6618 - accuracy: 0.76 - ETA: 1s - loss: 0.6531 - accuracy: 0.76 - ETA: 1s - loss: 0.6394 - accuracy: 0.76 - ETA: 1s - loss: 0.6285 - accuracy: 0.77 - ETA: 1s - loss: 0.6203 - accuracy: 0.77 - ETA: 1s - loss: 0.6124 - accuracy: 0.77 - ETA: 1s - loss: 0.6068 - accuracy: 0.78 - ETA: 1s - loss: 0.6008 - accuracy: 0.78 - ETA: 1s - loss: 0.5982 - accuracy: 0.78 - ETA: 1s - loss: 0.591

KeyboardInterrupt: 

In [ ]:
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [ ]:
# 결과물
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))